In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import math
from functools import partial
from tqdm import tqdm
import time

import bases
import eigensolvers
import functionals
import generators
import representations
import reconstructions
import utils
from test_base import TestNDTorus

In [2]:
def compare_plot_on_unit_square(data1, data2):
    assert data1.ndim == 2
    assert data2.ndim == 2
    N = len(data1)
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    utils.set_subplot_complex_on_unit_circle(data1, axs[0])
    utils.set_subplot_complex_on_unit_circle(data2, axs[1])
    plt.show()
    return

In [3]:
kwargs = {
    'n_dim': 1,
    'degree': 100,
    'num_col': 10**4,
}
tnd = TestNDTorus(**kwargs)
K, V, L = tnd._get_matrix()
eigenvalues, true_V, _ = eigensolvers.QR_algorithm_with_inverse_iteration(K)
_, rpower_V, _ = eigensolvers.QR_algorithm_with_power_vector(K, tnd.basis, tnd.Xr.shape[1])
iL, ipower_V, _, _ = eigensolvers.inverse_iteration_with_integer_power(K, tnd.basis, n_dim=tnd.Xr.shape[1], n_inv=2, eigenvalue_approx=eigenvalues)
mp_KM = reconstructions.get_koopman_modes(tnd.Xr, np.linalg.inv(V), tnd.basis)
tr_KM = reconstructions.get_koopman_modes(tnd.Xr, np.linalg.inv(true_V), tnd.basis)
rpo_KM = reconstructions.get_koopman_modes(tnd.Xr, np.linalg.inv(rpower_V), tnd.basis)
ipo_KM = reconstructions.get_koopman_modes(tnd.Xr, np.linalg.inv(ipower_V), tnd.basis)
mpPh = tnd.basis(tnd.Xr[0])@V
trPh = tnd.basis(tnd.Xr[0])@true_V
rpoPh = tnd.basis(tnd.Xr[0])@rpower_V
ipoPh = tnd.basis(tnd.Xr[0])@ipower_V
M = tnd.Xr.shape[0]
mp_reconstruction = np.array([np.squeeze(mpPh @ np.diag(np.power(L, k)) @ mp_KM) for k in range(M)])
tr_reconstruction = np.array([np.squeeze(trPh @ np.diag(np.power(L, k)) @ tr_KM) for k in range(M)])
rpo_reconstruction = np.array([np.squeeze(rpoPh @ np.diag(np.power(L, k)) @ rpo_KM) for k in range(M)])
ipo_reconstruction = np.array([np.squeeze(ipoPh @ np.diag(np.power(iL, k)) @ ipo_KM) for k in range(M)])
original_data = np.squeeze(tnd.Xr)

In [4]:
np.linalg.norm(original_data - mp_reconstruction)/M

0.0015589558560897769

In [5]:
np.linalg.norm(original_data - tr_reconstruction)/M

0.02604238982815002

In [6]:
np.linalg.norm(original_data - rpo_reconstruction)/M

229856338962334.22

In [7]:
np.linalg.norm(original_data - ipo_reconstruction)/M

1104237.2150640406

In [8]:
r = L.shape[0]
results = []
for n_inv in range(1, r+1, 50):
    try:
        st = time.time()
        iL, ipower_V, _, performed_powers = eigensolvers.inverse_iteration_with_integer_power(K, tnd.basis, n_dim=tnd.Xr.shape[1], n_inv=n_inv, eigenvalue_approx=eigenvalues)
        ipo_KM = reconstructions.get_koopman_modes(tnd.Xr, np.linalg.inv(ipower_V), tnd.basis)
        ipoPh = tnd.basis(tnd.Xr[0])@ipower_V
        ipo_reconstruction = np.array([np.squeeze(ipoPh @ np.diag(np.power(iL, k)) @ ipo_KM) for k in range(M)])
        et = time.time()
        original_data = np.squeeze(tnd.Xr)
        error = np.linalg.norm(original_data - ipo_reconstruction)/M
        print(f"n_inv={n_inv}, norm={round(error, 2)}, time_taken={et-st}")
        results.append((n_inv, error, et-st))
    except AssertionError:
        print(f"n_inv={n_inv}, eigenvalue assertion error.")
        continue
    except np.linalg.LinAlgError:
        print(f"n_inv={n_inv}, matrix inverse error")

n_inv=1, norm=1549183.67, time_taken=192.41170978546143
n_inv=51, norm=260.34, time_taken=148.71408486366272
n_inv=101, norm=369.41, time_taken=106.30403089523315
n_inv=151, norm=590.58, time_taken=64.16887497901917
n_inv=201, norm=0.02, time_taken=23.667874097824097


In [9]:
results

[(1, 1549183.6694562233, 192.41170978546143),
 (51, 260.34133997217856, 148.71408486366272),
 (101, 369.40634214598765, 106.30403089523315),
 (151, 590.5787081091904, 64.16887497901917),
 (201, 0.024365611629635346, 23.667874097824097)]

In [10]:
pd.set_option("display.precision", 2)
pd.DataFrame.from_records(results)

,0,1,2
0,1,1.55e+06,192.41
1,51,2.60e+02,148.71
2,101,3.69e+02,106.30
3,151,5.91e+02,64.17
4,201,2.44e-02,23.67
